<a href="https://colab.research.google.com/github/EricAshby/EDA-and-Cleaning-Invoices/blob/main/TEDA1030_Mod6_project_EricAshby_09_08_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Cleaning and Exploratory Data Analysis on Invoices
Eric D. Ashby

##Introduction
The `invoices.csv` data set comes from a fictitious company called World Wide Importers. The data set is constructed as if created from input over multiple years through the use of multiple programs.

The goal of this analysis is to clean the data and provide executives with insights into that data, specifically providing anything worth including in their future business strategies.

##Purpose of Analysis
This project aims to produce a clean data set on which to perform an exploratory data analysis. Specific issues to be dealt with include:

*  Standardization errors
*  Business logic errors
*  Missing values
  -  through imputation, where possible
  -  through deletion, where necessary
*  Duplicate entries
*  Outliers

Upon completion of the analysis, this project aims to answer the following questions from the company:

*  "Which customer has the most invoices?"
*  "Which item has the highest total quantity sold?"
*  "Which item has generated the most total profit?"
*  "Which item do customers purchase the most quantity of, on average?"

In [ ]:
import pandas as pd
dtypes = {'LastEditedWhen' : 'str'} #specify data type for column 12 to avoid DTypeWarning
df = pd.read_csv('invoices.csv', dtype = dtypes)

##Overview

Displayed below are the first 5 entries in the data set. Looking at these first few rows can be a useful way of spotting dirty data early on. Since our goal first requires us to clean this data, we may take note of a few particulars. To start, the Description and LestEditedWhen columns seem to contain multiple kinds of information. We may also notice some missing values and some negative values that don't make sense in context.

In [ ]:
df.head()

,InvoiceLineID,InvoiceID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy,LastEditedWhen,Customer
0,1,1,67.0,Ride on toy sedan car (Black) 1/12 scale,7,10,230.0,15,345.0,850.0,2645.0,7,NaN,Stuff by Stew
1,2,2,50.0,Developer joke mug - old C developers never di...,7,9,13.0,15,NaN,76.5,-999.0,7,1/1/2013 12:00,Wholesaler Plus
2,3,2,10.0,USB food flash drive - chocolate bar,7,9,32.0,15,43.2,180.0,331.2,7,NaN,Wholesaler Plus
3,4,3,114.0,Superhero action jacket (Blue) XXL,7,3,30.0,15,NaN,24.0,-999.0,7,NaN,Big Buys Retail
4,5,4,206.0,Permanent marker black 5mm nib (Black) 5mm,7,96,2.7,15,NaN,96.0,NaN,7,1/1/2013 12:00,Terry's Trinkets


If we look at the metadata (below), we can see the missing values, also called null values, are contained in the StockItemID, Description, TaxAmount, Extended Price, and—most notably—the LastEditedWhen columns.

We may also note that the StockItemID is stored, unusually, as a floating point number indead of the conventional integer.  This may or may not be relevant, though it is non-standard and may warrant some attention.

There are 230,548 entries in the data set with 14 columns of data.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230548 entries, 0 to 230547
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   InvoiceLineID   230548 non-null  int64  
 1   InvoiceID       230548 non-null  int64  
 2   StockItemID     230087 non-null  float64
 3   Description     229876 non-null  object 
 4   PackageTypeID   230548 non-null  int64  
 5   Quantity        230548 non-null  int64  
 6   UnitPrice       230548 non-null  float64
 7   TaxRate         230548 non-null  int64  
 8   TaxAmount       109623 non-null  float64
 9   LineProfit      230548 non-null  float64
 10  ExtendedPrice   113081 non-null  float64
 11  LastEditedBy    230548 non-null  int64  
 12  LastEditedWhen  6 non-null       object 
 13  Customer        230548 non-null  object 
dtypes: float64(5), int64(6), object(3)
memory usage: 24.6+ MB


The code below displays the descriptive statistics for the numeric data in the data set. Notice the negative values in the minima of several columns (TaxRate and ExtendedPrice in particular). These do not make sense in the context of their columns and will need to be addressed.

Also worth looking at are the maxima, particularly in the Quantity column. Large maxima in other columns *may* be explainable through large quantities of product ordered. However, the large quantity itself remains unexplained.  Considering the significant difference between the 75th quartile (60) and the maximum (3150), it seems we have at least one clear outlier in the data set.

Lastly, if we check the InvoiceLineID column, we see that the total number of entries (230548) exceeds the maximum invoice line ID (228265).  From this, we can conclude that there must be some entries with the same invoice line ID. That is to say, there may be duplicates in the data.

In [ ]:
df.describe()

,InvoiceLineID,InvoiceID,StockItemID,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy
count,230548.000000,230548.000000,230087.000000,230548.000000,230548.000000,230548.000000,230548.000000,109623.000000,230548.00000,113081.000000,230548.000000
mean,114127.066880,35177.386891,110.181827,7.073638,39.252524,45.539361,14.463409,113.328006,375.38894,740.092626,10.800480
std,65900.142929,20338.920057,63.722514,0.644528,55.193934,139.510031,3.904056,221.866696,753.17041,1705.229089,5.509623
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.660000,-15.000000,0.380000,-645.00000,-999.000000,2.000000
25%,57062.750000,17570.000000,54.000000,7.000000,5.000000,13.000000,15.000000,14.400000,51.00000,89.700000,6.000000
50%,114130.500000,35151.000000,111.000000,7.000000,10.000000,18.000000,15.000000,34.500000,120.00000,248.400000,11.000000
75%,171203.250000,52766.000000,165.000000,7.000000,60.000000,32.000000,15.000000,129.600000,390.00000,919.080000,16.000000
max,228265.000000,70510.000000,227.000000,10.000000,3150.000000,1899.000000,15.000000,14994.000000,9200.00000,114954.000000,20.000000


##Analysis

###Standardization Errors

####Customer Column

In looking for standardization errors, it is often productive to check in columns containing text data. Specifically, since one of the questions in our goal asks how many invoices each customer has, let's look at the Customer column in the data set.

The following code shows all unique entires in the Customer column:

In [ ]:
print(df['Customer'].unique())

['Stuff by Stew' 'Wholesaler Plus' 'Big Buys Retail' "Terry's Trinkets"
 'Stuf by Stew' 'stuff by stew' "Terry's Trinket" 'Dennis Distribution']


As can be seen above, we have some duplicate customers due to typos and differneces in capitalization.  First we will deal with with the the typos with the code below. This code replaces specified typos with the corrected version.

In [ ]:
df['Customer'] = df['Customer'].str.replace('Stuf ', 'Stuff ').str.replace('Trinket$', 'Trinkets', regex = True)

Second, we will take care of the capitalization problem with the following code which converts each customer name into title case.

In [ ]:
df['Customer'] = df['Customer'].str.title()

Here is our new list of unique customers:

In [ ]:
print(df['Customer'].unique())

['Stuff By Stew' 'Wholesaler Plus' 'Big Buys Retail' "Terry'S Trinkets"
 'Dennis Distribution']


Though it is not necessary (since the names are now standardized), for improved legibility, the following code fixes some peculiarities in the capitalization caused by the title case pass. The code then redisplays our customer list.

In [ ]:
df['Customer'] = df['Customer'].str.replace('\'S', '\'s').str.replace(' By ', ' by ')
print(df['Customer'].unique())

['Stuff by Stew' 'Wholesaler Plus' 'Big Buys Retail' "Terry's Trinkets"
 'Dennis Distribution']


###"Which customer has the most invoices?"

As previously touched on, one of our questions to be answered is "Which customer has the most invoices?" To answer this, we need to consider that each entry represents an invoice line. There can be multiple invoice lines in each invoice. Therefore, we must only count one entry from each unique invoice. One of the quickest ways to do this is to simply drop all entries with duplicate Customer and InvoiceID, leaving us with only the first invoice line in each unique invoice. We can then simply count up the invoices for each customer and report the results. The code below accomplishes just this.

In [ ]:
df[['InvoiceID', 'Customer']].drop_duplicates(subset = ['Customer', 'InvoiceID'])['Customer'].value_counts()

Stuff by Stew          31198
Terry's Trinkets       29025
Wholesaler Plus         4814
Big Buys Retail         4097
Dennis Distribution     1376
Name: Customer, dtype: int64

We can conclude from this that the customer **Stuff by Stew generated the most unique invoices**.

###Missing Values

####LastEditedWhen Column

In terms of missing values, we found in the overview that there are a few columns worth investigating. The code here highlights them specifically by counting the total number of null values in each column:

In [ ]:
print('Total Number of Null Vaules by Column:\n')
print(df.isna().sum())

Total Number of Null Vaules by Column:

InvoiceLineID          0
InvoiceID              0
StockItemID          461
Description          672
PackageTypeID          0
Quantity               0
UnitPrice              0
TaxRate                0
TaxAmount         120925
LineProfit             0
ExtendedPrice     117467
LastEditedBy           0
LastEditedWhen    230542
Customer               0
dtype: int64


For a look at it from a proportunal point of view:

In [ ]:
print('Percentage of Entires Containing Null Vaules by Column:\n')
print(df.isna().mean() * 100) #multiplied by 100 to get percentage form

Percentage of Entires Containing Null Vaules by Column:

InvoiceLineID      0.000000
InvoiceID          0.000000
StockItemID        0.199958
Description        0.291479
PackageTypeID      0.000000
Quantity           0.000000
UnitPrice          0.000000
TaxRate            0.000000
TaxAmount         52.451116
LineProfit         0.000000
ExtendedPrice     50.951212
LastEditedBy       0.000000
LastEditedWhen    99.997398
Customer           0.000000
dtype: float64


From this, we can clearly see that the LastEditedWhen column is almost entirely empty. This data is not imputable and doesn't have any bearing on the questions posed in this analysis. The best course of action, then, would be to drop the column entirely.

The code below removes the LastEditedWhen column from the data set.

In [ ]:
df = df.drop(columns = ['LastEditedWhen'])

As can be seen below, the LastEditedWhen column is no longer included in the data set and the new leader in terms of null values is the TaxAmount column. Luckily, this data is imputable using other information in the data set. However, this will have to wait until later due to some issues with the TaxRate column discovered in the overview and which will be discussed in the Business Logic Errors section.

In [ ]:
print('Total Number of Null Vaules by Column:\n')
print(df.isna().sum())

Total Number of Null Vaules by Column:

InvoiceLineID         0
InvoiceID             0
StockItemID         461
Description         672
PackageTypeID         0
Quantity              0
UnitPrice             0
TaxRate               0
TaxAmount        120925
LineProfit            0
ExtendedPrice    117467
LastEditedBy          0
Customer              0
dtype: int64


####StockItemID and Description

#####*Imputing Missing Descriptions*

Previously, we determined that both the StockItemID and Description columns contain null values. Since the product description is specific to the stock item ID, we can mitigate these null values through imputation: finding stock item IDs with descriptions and filling in those same descriptions into other instances of the same item.

The code below shows, in ascending order, all the unique stock item IDs present in the data set. There are 227 unique IDs plus one for entries with a missing stock ID (nan).

In [ ]:
stockItemIDs = df['StockItemID'].sort_values().unique()
stockItemIDs

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

Conveniently, if we check the unique descriptions with the following code, we find that there the same number of descriptions in the data set.

In [ ]:
#find all unique descriptions
descriptions = df['Description'].sort_values().unique()

#report the number of descriptions
print(
    'There are',
    len(descriptions),
    'unique descriptions in the data set, the',
    str(len(descriptions)) + 'th',
    'of which is',
    descriptions[len(descriptions) - 1]
    )


There are 228 unique descriptions in the data set, the 228th of which is nan


Also quite convenient, the stock item IDs are integers that go from 1 to 227 without skipping any numbers. So, we can use them as indeces in a list, where the indeces are the stock item IDs and the values are the associated descriptions. The code below creates such a list and fills in each of the descriptions for which we have a stock item ID.

Updatated counts for null values are displayed.

In [ ]:
# df[['StockItemID', 'Description']].groupby('StockItemID').value_counts()
df_IDdescription = df[['StockItemID', 'Description']].sort_values(by = 'StockItemID').drop_duplicates(subset = 'StockItemID')

#create properly indexed list of descriptions
IDdescription_list = [None] #initialized to have a null first value so as to have ideces line up with stock item IDs
for i, entry in df_IDdescription.iterrows():
  IDdescription_list.append(entry[1])

IDdescription_list.pop() #remove the last entry (nan-nan stockItemID-Description pair)

#fill in missing descriptions
for i in range(len(IDdescription_list)):
  #create filter for StockItemIDs equal to current index i
  if i != 0:
    StockItemFilter = (df['StockItemID'] == i)
    df.loc[StockItemFilter, 'Description'] = df.loc[StockItemFilter, 'Description'].fillna(IDdescription_list[i])

print('Total Number of Null Vaules by Column:\n')
print(df.isna().sum())

Total Number of Null Vaules by Column:

InvoiceLineID         0
InvoiceID             0
StockItemID         461
Description         461
PackageTypeID         0
Quantity              0
UnitPrice             0
TaxRate               0
TaxAmount        120925
LineProfit            0
ExtendedPrice    117467
LastEditedBy          0
Customer              0
dtype: int64


#####*Remaining Null Values in StockItemID and Description*

Unfortunately, there still remain null values in both columns. There are 461 entries for which there are missing values in both the StockItemID and Description columns.  This can bee seen explicitly with the following code:

In [ ]:
#create filter for both columns containing null values
bothNull = (df['StockItemID'].isna()) & (df['Description'].isna())

#determine how many entries have both columns empty and report
df[bothNull]

,InvoiceLineID,InvoiceID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy,Customer
596,597,251,NaN,NaN,7,1,13.0,15,NaN,8.5,NaN,3,Stuff by Stew
1667,1668,587,NaN,NaN,7,5,32.0,15,24.0,120.0,184.0,19,Stuff by Stew
1981,1982,674,NaN,NaN,7,7,32.0,15,33.6,168.0,257.6,5,Stuff by Stew
2324,2325,775,NaN,NaN,7,100,32.0,15,480.0,1600.0,3680.0,9,Terry's Trinkets
2792,2793,919,NaN,NaN,9,7,240.0,15,NaN,1060.5,NaN,17,Terry's Trinkets
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227821,227822,70381,NaN,NaN,7,8,32.0,15,NaN,192.0,NaN,6,Stuff by Stew
227909,227910,70407,NaN,NaN,7,2,13.0,15,3.9,17.0,29.9,6,Stuff by Stew
228538,167737,51705,NaN,NaN,7,84,18.0,-15,226.8,924.0,1738.8,5,Wholesaler Plus
229337,219123,67693,NaN,NaN,7,10,13.0,15,19.5,85.0,149.5,19,Terry's Trinkets


However, considering that the remaining questions ask about items specifically, we find that the remaining entries are not usful in answering these questions since the products cannot be identified (due to their stock item IDs not being imputable). Additionally, we may note that 461 entries is quite small in comparison to the full data set. We will then drop the offending entries from the data set.

In [ ]:
df = df.drop(index = df[bothNull].index)
df

,InvoiceLineID,InvoiceID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy,Customer
0,1,1,67.0,Ride on toy sedan car (Black) 1/12 scale,7,10,230.00,15,345.00,850.0,2645.00,7,Stuff by Stew
1,2,2,50.0,Developer joke mug - old C developers never di...,7,9,13.00,15,NaN,76.5,-999.00,7,Wholesaler Plus
2,3,2,10.0,USB food flash drive - chocolate bar,7,9,32.00,15,43.20,180.0,331.20,7,Wholesaler Plus
3,4,3,114.0,Superhero action jacket (Blue) XXL,7,3,30.00,15,NaN,24.0,-999.00,7,Big Buys Retail
4,5,4,206.0,Permanent marker black 5mm nib (Black) 5mm,7,96,2.70,15,NaN,96.0,NaN,7,Terry's Trinkets
...,...,...,...,...,...,...,...,...,...,...,...,...,...
230543,36558,11285,139.0,Furry animal socks (Pink) M,10,36,5.00,15,27.00,126.0,207.00,20,Wholesaler Plus
230544,21110,6502,9.0,USB food flash drive - banana,7,2,32.00,15,NaN,40.0,NaN,10,Stuff by Stew
230545,34087,10504,179.0,Shipping carton (Brown) 229x229x229mm,7,125,1.05,15,NaN,62.5,NaN,16,Terry's Trinkets
230546,103435,31836,20.0,DBA joke mug - you might be a DBA if (White),7,1,13.00,15,1.95,8.5,14.95,20,Big Buys Retail


###Busisness Logic Errors

####TaxRate Column

As was briefly mentioned in the overview and the section dealing with the LastEditedWhen column, there is a business logic error in the TaxRate column. This error is clearly visible in the descriptive statistics of the data set. The following code shows us the descriptive statistics for the TaxRate column (as it is presently comprised).

In [ ]:
df['TaxRate'].describe()

count    230087.000000
mean         14.463268
std           3.904501
min         -15.000000
25%          15.000000
50%          15.000000
75%          15.000000
max          15.000000
Name: TaxRate, dtype: float64

A negative value is nonsensical in the context of tax rates. This is clearly an error, though we are not yet sure if it is the only error. The code, here, displays the number of entries with negative tax rates.

In [ ]:
#create filter for negative tax rates
negTaxRate = (df['TaxRate'] < 0)

#display the total number of negative tax rates (i.e. the number of trues in the filter)
negTaxRate.sum()

3946

We find there to be almost 4000 entries with negative tax rates. Outside of this, the entries here do not seem to have a lot in common. It seems safe to say, then, that the negative tax rates are simply mis-inputs and not due to some systematic error.

We resolve this issue by simply converting all negative entries in the TaxRate columns into positive ones. The code below accomplishes this.

In [ ]:
df.loc[negTaxRate, 'TaxRate'] *= -1

A quick check on the descriptive statistics (displayed by the code below) shows us that all the negative tax rates have been corrected, shifting the average taxe rate slightly higher and leaving us with a new minumum of 10.

In [ ]:
df['TaxRate'].describe()

count    230087.000000
mean         14.977248
std           0.336518
min          10.000000
25%          15.000000
50%          15.000000
75%          15.000000
max          15.000000
Name: TaxRate, dtype: float64

###Missing Values (continued)

####TaxAmount Column

With the tax rate fixed, we are now free to impute the missing TaxAmount values. The tax amount is calculated with the formula:

`TaxAmount = (TaxRate / 100) * UnitPrice * Quantity`

Here, TaxRate must be divided by 100 as it is stored in percentage form in the data set.


The folowing code fills empty entries in the TaxAmount column using the above formula.

In [ ]:
df['TaxAmount'] = df['TaxAmount'].fillna((df['TaxRate'] / 100) * df['UnitPrice'] *df['Quantity'])

Displayed below are the new descriptive statistics for the, now corrected, TaxAmount column of the data set.

In [ ]:
df['TaxAmount'].describe()

count    230087.000000
mean        112.952167
std         219.486804
min           0.375000
25%          14.400000
50%          34.500000
75%         129.600000
max       14994.000000
Name: TaxAmount, dtype: float64

####ExtendedPrice Column



Now that we've taken care of several of the columns with missing data, let's take a look once again at the number of null values remaining in the data set.

In [ ]:
print('Total Number of Null Vaules by Column:\n')
print(df.isna().sum())

Total Number of Null Vaules by Column:

InvoiceLineID         0
InvoiceID             0
StockItemID           0
Description           0
PackageTypeID         0
Quantity              0
UnitPrice             0
TaxRate               0
TaxAmount             0
LineProfit            0
ExtendedPrice    117229
LastEditedBy          0
Customer              0
dtype: int64


We see that our last column to take care of (in terms of missing values) is ExtendedPrice.  This value can be imputed with the existing information in the data set.  Extended price is calculated using the formula:

`ExtendedPrice = (UnitPrice * Quantity) + TaxAmount`

Similarly as the previous section, the following code in-fills the missing data using this formula.

In [ ]:
df['ExtendedPrice'] = df['ExtendedPrice'].fillna((df['UnitPrice'] * df['Quantity']) + df['TaxAmount'])

This leaves us with new descriptive statistics for extented price, shown by the code below.

In [ ]:
df['ExtendedPrice'].describe()

count    230087.000000
mean        803.721183
std        1688.000979
min        -999.000000
25%         104.650000
50%         257.600000
75%         993.600000
max      114954.000000
Name: ExtendedPrice, dtype: float64

Note the minimum displayed above. We know that the UnitPrice, Quantity, and TaxAmount columns contain no negative values as seen in the overview. A negative value in the ExtendedPrice column could only have been in error.

Evidently, there are errors in the ExtendedPrice column outside of just missing values. This is easily remedied, however, as we have the information necessary to impute all the extended prices in the data set.

The code below imputes the extended price for *all entries* and corrects such for the entire data set.

In [ ]:
df['ExtendedPrice'] = (df['UnitPrice'] * df['Quantity']) + df['TaxAmount']

Our new (now fully corrected) descriptive statistics are displayed below.

In [ ]:
df['ExtendedPrice'].describe()

count    230087.000000
mean        867.637314
std        1683.168093
min           2.875000
25%         110.400000
50%         264.500000
75%         993.600000
max      114954.000000
Name: ExtendedPrice, dtype: float64

###Outliers

####Quantity and ExtendedPrice

In the overview, we saw that there were some maxima in the descriptive statistics that seemed rather high. Below, this code selects two such columns that are particularly suspect and displays their descripive statistics.

In [ ]:
df[['Quantity', 'ExtendedPrice']].describe()

,Quantity,ExtendedPrice
count,230087.000000,230087.000000
mean,39.247267,867.637314
std,55.193231,1683.168093
min,1.000000,2.875000
25%,5.000000,110.400000
50%,10.000000,264.500000
75%,60.000000,993.600000
max,3150.000000,114954.000000


Both look severely right skewed, judging from their medians and means. The code below creates columns for the z-scores of these two columns.

In [ ]:
#calculate means
avgQ = df['Quantity'].mean()
avgEP = df['ExtendedPrice'].mean()

#calculate standard deviations
stdQ = df['Quantity'].std()
stdEP = df['ExtendedPrice'].std()

#create columns with calculated z-scores
df['Zscore_Q'] = (df['Quantity'] - avgQ) / stdQ
df['Zscore_EP'] = (df['ExtendedPrice'] - avgEP) / stdEP

We will choose a standard of z = 3 to define ouliers.  That is, entires with z-scores equal to or greater than 3 are considered outliers. Additionally, entries with z-scores less than or equal to -3 are also considered outliers.

This code identifies and displays the entries containing outliers in the Quantity and/or ExtendedPrice columns:

In [ ]:
#create filter for outliers
outliers = (df['Zscore_Q'].apply('abs') >= 3) | (df['Zscore_EP'].apply('abs') >= 3) #use absolute value to check both >=3 and <=-3 at the same time

#display ouliers
df[outliers]

,InvoiceLineID,InvoiceID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy,Customer,Zscore_Q,Zscore_EP
96,97,45,164.0,32 mm Double sided bubble wrap 50m,7,50,112.0,15,840.00,2650.0,6440.00,9,Stuff by Stew,0.194820,3.310639
111,112,53,170.0,20 mm Anti static bubble wrap (Blue) 50m,7,90,102.0,15,1377.00,4230.0,10557.00,9,Stuff by Stew,0.919546,5.756622
122,123,59,191.0,Black and orange fragile despatch tape 48mmx75m,7,288,3.7,15,159.84,547.2,1225.44,9,Terry's Trinkets,4.506943,0.212577
124,125,59,167.0,10 mm Anti static bubble wrap (Blue) 50m,7,60,99.0,15,891.00,3240.0,6831.00,9,Terry's Trinkets,0.376001,3.542939
197,198,95,191.0,Black and orange fragile despatch tape 48mmx75m,7,252,3.7,15,139.86,478.8,1072.26,9,Stuff by Stew,3.854689,0.121570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230407,209037,64540,158.0,10 mm Double sided bubble wrap 50m,7,100,105.0,15,1575.00,4900.0,12075.00,17,Dennis Distribution,1.100728,6.658493
230458,151681,46729,195.0,Black and orange handle with care despatch tap...,7,216,3.7,15,119.88,518.4,919.08,5,Terry's Trinkets,3.202435,0.030563
230475,217008,67029,201.0,Red and white urgent despatch tape 48mmx75m,7,240,3.7,15,133.20,504.0,1021.20,18,Stuff by Stew,3.637271,0.091234
230483,1708,598,215.0,Air cushion machine (Blue),7,5,1899.0,15,1424.25,3795.0,10919.25,15,Terry's Trinkets,-0.620498,5.971841


In the context of the size of the data set, under 11 thousand outliers is not entirely unreasonable. The code below shows the percentage of the whole data set that are considered outliers: less than 5%.

In [ ]:
[entries, col] = df.shape
print(str(round(outliers.sum() / entries * 100,2)) + '%')

4.57%


Specifically, we find that 5987 entries are outliers in quantity and 4522 entries are ouliers in extended price (with some small overlap). This is shown by the following code:

In [ ]:
#create filter for outliers
outliers_Q = (df['Zscore_Q'].apply('abs') >= 3) #use absolute value to check both >=3 and <=-3 at the same time
outliers_EP = (df['Zscore_EP'].apply('abs') >= 3)
outliers_overlap = outliers_Q & outliers_EP

#display ouliers
print('Number of Outliers:\n')
print('Quantity:', outliers_Q.sum())
print('ExtendedPrice:', outliers_EP.sum())
print('Overlap:', outliers_overlap.sum(0))

Number of Outliers:

Quantity: 5987
ExtendedPrice: 4522
Overlap: 5


The three remaining questions are concerned with the extremes in quantity and profit (which strongly correlates with extended price as demonstrated by the code below). As such, it would be inappropriate to remove the exact data the questions are about. Therefore, it will suffice for this analysis to note that these outliers exist but to also leave them intact.

In [ ]:
df[['LineProfit', 'ExtendedPrice']].corr()

,LineProfit,ExtendedPrice
LineProfit,1.000000,0.943862
ExtendedPrice,0.943862,1.000000


###Duplicates

####InvoiceLineID Column

In the overview, we determined from the maximum invoice line ID that there must be duplicate entries in the data set. The code below identifies and displays the entries with duplicate invoice line IDs.  We find over 2000 such duplicates.

In [ ]:
df[df.duplicated(subset = ['InvoiceLineID'])]

,InvoiceLineID,InvoiceID,StockItemID,Description,PackageTypeID,Quantity,UnitPrice,TaxRate,TaxAmount,LineProfit,ExtendedPrice,LastEditedBy,Customer,Zscore_Q,Zscore_EP
228265,199860,61652,222.0,Chocolate beetles 250g,1,216,8.55,10,184.6800,820.8,2031.4800,5,Stuff by Stew,3.202435,0.691460
228266,197512,60915,161.0,20 mm Double sided bubble wrap 50m,7,50,108.00,15,810.0000,4600.0,6210.0000,12,Stuff by Stew,0.194820,3.173992
228267,113602,34985,167.0,10 mm Anti static bubble wrap (Blue) 50m,7,40,99.00,15,594.0000,2160.0,4554.0000,4,Stuff by Stew,0.013638,2.190133
228268,6306,1950,184.0,Shipping carton (Brown) 305x305x305mm,7,100,3.50,15,52.5000,160.0,402.5000,16,Stuff by Stew,1.100728,-0.276346
228269,134380,41395,155.0,Large sized bubblewrap roll 50m,7,50,24.00,15,180.0000,550.0,1380.0000,20,Stuff by Stew,0.194820,0.304404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230543,36558,11285,139.0,Furry animal socks (Pink) M,10,36,5.00,15,27.0000,126.0,207.0000,20,Wholesaler Plus,-0.058835,-0.392496
230544,21110,6502,9.0,USB food flash drive - banana,7,2,32.00,15,9.6000,40.0,73.6000,10,Stuff by Stew,-0.674852,-0.471752
230545,34087,10504,179.0,Shipping carton (Brown) 229x229x229mm,7,125,1.05,15,19.6875,62.5,150.9375,16,Terry's Trinkets,1.553682,-0.425804
230546,103435,31836,20.0,DBA joke mug - you might be a DBA if (White),7,1,13.00,15,1.9500,8.5,14.9500,20,Big Buys Retail,-0.692970,-0.506597


Since we know that invoice line IDs are unique to each transaction, we also know that these entries must be duplicates, perhaps a result of combining data sets in the past. As determining which duplicate should be kept would have to be done on a case by case basis, it is not reasonable to be so selective. Instead, the following code drops all instances of each duplicate after the first.

In [ ]:
df = df.drop_duplicates(subset = ['InvoiceLineID'])

This leaves us with 227,811 remaining entries in the data set which is shown in the metadata displayed by the code below.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227811 entries, 0 to 229321
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   InvoiceLineID  227811 non-null  int64  
 1   InvoiceID      227811 non-null  int64  
 2   StockItemID    227811 non-null  float64
 3   Description    227811 non-null  object 
 4   PackageTypeID  227811 non-null  int64  
 5   Quantity       227811 non-null  int64  
 6   UnitPrice      227811 non-null  float64
 7   TaxRate        227811 non-null  int64  
 8   TaxAmount      227811 non-null  float64
 9   LineProfit     227811 non-null  float64
 10  ExtendedPrice  227811 non-null  float64
 11  LastEditedBy   227811 non-null  int64  
 12  Customer       227811 non-null  object 
 13  Zscore_Q       227811 non-null  float64
 14  Zscore_EP      227811 non-null  float64
dtypes: float64(7), int64(6), object(2)
memory usage: 27.8+ MB


###"Which item has the highest total quantity sold?"

Now, with our cleaned data, we are free to analyze it and answer the remaining questions. To determine which item had the highest total quantity sold, we will use the Description and Quantity columns in the following code:

In [ ]:
df[['Description', 'Quantity']].groupby('Description').sum().sort_values(by = 'Quantity', ascending = False).head()

,Quantity
Description,
Black and orange fragile despatch tape 48mmx75m,207108
Black and orange fragile despatch tape 48mmx100m,193680
Clear packaging tape 48mmx75m,158574
3 kg Courier post bag (White) 300x190x95mm,155050
Shipping carton (Brown) 356x356x279mm,151825


The code above shows the the total quantity sold for the top five best selling products. We find that **'Black and orange fragile despatch tape 48mmx75m' was the highest quantity item sold**.

###"Which item has generated the most total profit?"

To answer this question, we can use the Description and LineProfit columns to generate a table showing the total profits from each product. The following code shows this for the top five most profitable products, sorted from greatest to least profits.

In [ ]:
df[['Description', 'LineProfit']].groupby('Description').sum().sort_values(by = 'LineProfit', ascending = False).head()

,LineProfit
Description,
20 mm Double sided bubble wrap 50m,5283560.0
Air cushion machine (Blue),4426488.0
32 mm Anti static bubble wrap (Blue) 50m,3526400.0
10 mm Anti static bubble wrap (Blue) 50m,3439800.0
32 mm Double sided bubble wrap 50m,2915530.0


We conclude that **the '20 mm Double sided bubble wrap 50m' has generated the most profit**.

###"Which item do customers purchase the most quantity of, on average?"

Finally, we answer the last question with the Description and Quantity column once again.

In [ ]:
df[['Description', 'Quantity']].groupby('Description').mean().sort_values(by = 'Quantity', ascending = False).head()

,Quantity
Description,
Black and orange fragile despatch tape 48mmx75m,199.526012
Black and orange fragile despatch tape 48mmx100m,199.054471
Clear packaging tape 48mmx75m,145.347388
3 kg Courier post bag (White) 300x190x95mm,144.098513
Shipping carton (Brown) 356x356x279mm,141.627799


The preceding code finds the average quantity sold in a single invoice for the top 5 best bulk-selling products. We find that **customers purchase 'Black and orange fragile despatch tape 48mmx75m' in the highest quantities on average**.

##Results
This data cleaning and subsequent data analysis found that *Stuff by Stew generated the most unique invoices*. After further cleaning the data of problematic entries that were no longer needed for analysis, we also found that *'Black and orange fragile despatch tape 48mmx75m'* was the highest quantity item sold, the *'20 mm Double sided bubble wrap 50m'* has generated the most profit, and customers purchase *'Black and orange fragile despatch tape 48mmx75m'* in the highest quantities on average, keeping in mind the existence of outliers as identified in the outliers section. These outliers were left intact as they were relevant or rather *key* to answering the three remaining questions.